# Домашнее задание № 2. Мешок слов

In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from razdel import tokenize
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier as RandFor
from sklearn.tree import DecisionTreeClassifier as DecTree
from sklearn.naive_bayes import MultinomialNB as NaiveB
from string import punctuation as p

In [2]:
data = pd.read_csv('labeled.csv')

In [3]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [4]:
train.head(3)

,index,comment,toxic
0,6942,У нас до сих пор по городу такие тралики ходят\n,0.0
1,7417,"Интересно, как на ней происходит корректировка...",0.0
2,5503,Было у меня нечто похожее.Тоже на работе сидел...,0.0


## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [16]:
# векторизаторы бывают разные (bm25 хотя бы), tfidf по лекции
default_tfidf_vectorizer = TfidfVectorizer(
    min_df=5, max_df=0.4)

def razdel_tokenize(text):
    return [token.text for token in list(tokenize(text))]

custom_tfidf_vectorizer = TfidfVectorizer(
    tokenizer=razdel_tokenize, min_df=5, max_df=0.4)

In [29]:
x_train_1 = default_tfidf_vectorizer.fit_transform(train.comment)
x_train_2 = custom_tfidf_vectorizer.fit_transform(train.comment)
y_train = train.toxic.values

default_tfidf_vectorizer.get_feature_names_out()

array(['00', '000', '01', ..., 'ясно', 'ящик', 'ёмкости'], dtype=object)

In [30]:
x_test_1 = default_tfidf_vectorizer.transform(test.comment)
x_test_2 = custom_tfidf_vectorizer.transform(test.comment)
y_test = test.toxic.values

In [35]:
model_1 = LogReg()
model_1.fit(x_train_1, y_train)
preds_1 = model_1.predict(x_test_1)
print(classification_report(y_test, preds_1, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.82      0.95      0.88       923
         1.0       0.88      0.63      0.74       519

    accuracy                           0.84      1442
   macro avg       0.85      0.79      0.81      1442
weighted avg       0.84      0.84      0.83      1442



In [36]:
model_2 = LogReg()
model_2.fit(x_train_2, y_train)
preds_2 = model_2.predict(x_test_2)
print(classification_report(y_test, preds_2, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.83      0.95      0.88       923
         1.0       0.88      0.65      0.75       519

    accuracy                           0.84      1442
   macro avg       0.85      0.80      0.81      1442
weighted avg       0.85      0.84      0.83      1442



В этом эксперименте кастомный векторизатор оказался лучше по полноте на 0,02 в случае токсичных и хуже по точности на 0,01 в случае нормальных, в итоге f1 у кастомного для токсичных лучше на 0,01, но в целом метрики практически одинаковые

## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [38]:
stops = stopwords.words("russian")

In [52]:
count_vectorizer = CountVectorizer(
    tokenizer=razdel_tokenize, 
    stop_words=stops, 
    ngram_range=(1, 2), 
    min_df=5, 
    max_df=0.4)

In [51]:
tfidf_vectorizer = TfidfVectorizer(
    tokenizer=razdel_tokenize, 
    stop_words=stops, 
    ngram_range=(1, 2),
    min_df=5, 
    max_df=0.4)

In [69]:
x_train_1 = count_vectorizer.fit_transform(train.comment)
x_test_1 = count_vectorizer.transform(test.comment)

In [48]:
x_train_2 = tfidf_vectorizer.fit_transform(train.comment)
x_test_2 = tfidf_vectorizer.transform(test.comment)

In [44]:
y_train = train.toxic.values
y_test = test.toxic.values

In [71]:
model_1 = RandFor(n_estimators=120, criterion='entropy')
model_1.fit(x_train_1, y_train)
preds_1 = model_1.predict(x_test_1)
print(classification_report(y_test, preds_1, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.84      0.84      0.84       923
         1.0       0.72      0.71      0.72       519

    accuracy                           0.80      1442
   macro avg       0.78      0.78      0.78      1442
weighted avg       0.80      0.80      0.80      1442



In [46]:
model_2 = LogReg(C=0.1, class_weight='balanced')
model_2.fit(x_train_2, y_train)
preds_2 = model_2.predict(x_test_2)
print(classification_report(y_test, preds_2, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.88      0.84      0.86       923
         1.0       0.73      0.79      0.76       519

    accuracy                           0.82      1442
   macro avg       0.80      0.81      0.81      1442
weighted avg       0.82      0.82      0.82      1442



In [141]:
probs_1 = {}
for i, j in enumerate(model_1.predict_proba(x_test_1)):
    probs_1[i] = j[1]
# в первой колонке -- вероятности токсичных, так что j[1]
toxic_1 = sorted(probs_1.items(), key=lambda x: x[1], reverse=True)[:10]
print(toxic_1)
toxic_1 = [i[0] for i in toxic_1]

[(104, 1.0), (204, 1.0), (279, 1.0), (297, 1.0), (535, 1.0), (610, 1.0), (618, 1.0), (677, 1.0), (813, 1.0), (912, 1.0)]


In [142]:
probs_2 = {}
for i, j in enumerate(model_2.predict_proba(x_test_2)):
    probs_2[i] = j[1]
toxic_2 = sorted(probs_2.items(), key=lambda x: x[1], reverse=True)[:10]
print(toxic_2)
toxic_2 = [i[0] for i in toxic_2]

[(1088, 0.8624018524099621), (31, 0.8584254391870806), (102, 0.8584254391870806), (352, 0.8584254391870806), (437, 0.8584254391870806), (1415, 0.8584254391870806), (1076, 0.8071197438283689), (281, 0.799810291822574), (906, 0.7945795159552033), (985, 0.7932895680478177)]


In [145]:
for i in toxic_1:
    for j in toxic_2:
        if i == j:
            print(i)
else:
    print('No shared')

No shared


In [140]:
test['comment'].iloc[toxic_1].values

array(['Теперь хохлов бкдкт травить анашой, чтобы не пиздели на барина.\n',
       'Отписался от долбоеба, все подписки засрал дебил блять\n',
       'Иди нахуй,бля. Я искал эту хуйню так как дрочу в гостиницах, когда в командировки летаю. Сплю бухой без трусов. Мне нужная эта ебень.\n',
       'У хохлов теперь томос есть. То есть нихуя они не провославные, а сектанты ебанные и мрази. А у русни его нет и никогда не было. Учи матчасть, чтобы не быть батхёртом\n',
       'Отправить ты можешь только хуй в свою жопу, лахтодырка.\n',
       'если на борде не будет хохлов то не будет про них постов\n',
       'Какие же пиндосы дегенераты, пиздец просто.\n',
       'Канада пристанище куколдов хохлов, не удивительно что они рашку хейтят.\n',
       'То-то декоммунизированая свиноина процветает, пиздец просто',
       'На мотив Славянки. В жопу клюнул жареный петух...\n'],
      dtype=object)

In [144]:
test['comment'].iloc[toxic_2].values

array(['БЛЯТЬ! Опять ржу, как представлю минет вприкуску с пирогами! Чувак ты уникум!!!!!\n',
       'А у нас микроминимализм!\n', 'Пьяного-могила исправит!\n',
       'ты на стол-то ложись, русалка!\n',
       'Чтоб костями не колола, бл ать!\n',
       'азиатский окатыш, не бомби!\n', 'Замуж тебе надо барыня...\n',
       'А чё, нормуль! Проезду не мешает! Быдло mode off\n',
       'Майдауны ебучие.Их че хохлы там чтоли покусали? Пидары блядь\n',
       'КАКИЕ ХОХЛЫ?! два урода. при чём ту вообще ХОХЛЫ! аноны, смотрите! я двух шлюх проплаченых поймал!\n'],
      dtype=object)

Совпадающих текстов в топ-10 нет. Тексты действительно токсичны

## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию. 
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

In [98]:
stops = stopwords.words("russian") + [i for i in p] + \
['это', 'тебе', 'очень', 'просто', 'ещё', 'года', '2', '3']
# в принципе про "очень" можно поспорить, как и про другие

def razdel_tokenize(text):
    return [token.text for token in list(tokenize(text))]

In [99]:
vectorizer = TfidfVectorizer(
    tokenizer=razdel_tokenize, 
    stop_words=stops, 
    min_df=5, 
    max_df=0.4)

In [100]:
x_train = vectorizer.fit_transform(train.comment)
x_test = vectorizer.transform(test.comment)

In [10]:
y_train = train.toxic.values
y_test = test.toxic.values

Logistic Regression

In [101]:
model_1 = LogReg(C=0.1, class_weight='balanced')
model_1.fit(x_train, y_train)
preds_1 = model_1.predict(x_test)
print(classification_report(y_test, preds_1, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.91      0.85      0.88       951
         1.0       0.74      0.83      0.78       491

    accuracy                           0.84      1442
   macro avg       0.82      0.84      0.83      1442
weighted avg       0.85      0.84      0.84      1442



In [102]:
importance_1 = pd.DataFrame({
    'feature': vectorizer.get_feature_names_out(), 
    'importance': abs(model_1.coef_[0])
})
importance_1.sort_values('importance', ascending=False)[:10]

,feature,importance
7180,хохлы,1.530624
7178,хохлов,1.508734
3332,нахуй,1.270526
6193,спасибо,1.179985
445,блять,1.077598
444,блядь,1.060830
7201,хуй,1.039298
4247,пиздец,1.038297
6468,сука,0.963918
2006,знаю,0.960377


Decision Tree

In [103]:
model_2 = DecTree(criterion='entropy', class_weight='balanced')
model_2.fit(x_train, y_train)
preds_2 = model_2.predict(x_test)
print(classification_report(y_test, preds_2, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.84      0.78      0.81       951
         1.0       0.63      0.72      0.67       491

    accuracy                           0.76      1442
   macro avg       0.74      0.75      0.74      1442
weighted avg       0.77      0.76      0.77      1442



In [104]:
importance_2 = pd.DataFrame({
    'feature': vectorizer.get_feature_names_out(), 
    'importance': model_2.feature_importances_
})
importance_2.sort_values('importance', ascending=False)[:10]

,feature,importance
8,...,0.012237
7180,хохлы,0.010926
7178,хохлов,0.008795
3332,нахуй,0.008166
2006,знаю,0.007389
3274,например,0.006947
3574,нужно,0.005899
2726,лет,0.005872
6468,сука,0.005792
920,всё,0.005758


Naive Bayes

In [105]:
model_3 = NaiveB(alpha=0.5, fit_prior=False)
model_3.fit(x_train, y_train)
preds_3 = model_3.predict(x_test)
print(classification_report(y_test, preds_3, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.90      0.89      0.89       951
         1.0       0.79      0.80      0.80       491

    accuracy                           0.86      1442
   macro avg       0.84      0.85      0.84      1442
weighted avg       0.86      0.86      0.86      1442



In [106]:
importance_3 = pd.DataFrame({
    'feature': vectorizer.get_feature_names_out(), 
    'importance': model_3.feature_log_prob_[1]    
})
importance_3.sort_values('importance', ascending=False)[:10]

,feature,importance
8,...,-5.248865
7180,хохлы,-5.860254
7178,хохлов,-5.918063
4811,почему,-5.934659
3332,нахуй,-6.001342
7201,хуй,-6.209338
445,блять,-6.245257
4247,пиздец,-6.269382
444,блядь,-6.277723
920,всё,-6.301655


Random Forest

In [107]:
model_4 = RandFor(n_estimators=120, criterion='entropy')
model_4.fit(x_train, y_train)
preds_4 = model_1.predict(x_test)
print(classification_report(y_test, preds_4, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.91      0.85      0.88       951
         1.0       0.74      0.83      0.78       491

    accuracy                           0.84      1442
   macro avg       0.82      0.84      0.83      1442
weighted avg       0.85      0.84      0.84      1442



In [108]:
importance_4 = pd.DataFrame({
    'feature': vectorizer.get_feature_names_out(), 
    'importance': model_4.feature_importances_    
})
importance_4.sort_values('importance', ascending=False)[:10]

,feature,importance
7180,хохлы,0.007980
7178,хохлов,0.007889
3332,нахуй,0.005899
8,...,0.005414
4247,пиздец,0.004817
444,блядь,0.004607
6468,сука,0.004502
3274,например,0.004494
2006,знаю,0.004363
445,блять,0.004239


Вывожу по 10 самых токсичных, чтобы оценить, что там дальше, чем 5 (и не подкручиваю стоп-слова далее). Спорным может показаться токен "...", но, как кажется, он как раз вполне может говорить о токсичности. Нецензурные и оскорбительные слова происходят из исходных данных комментариев, сожалею